In [ ]:
import glob
import pathlib
import pickle
import numpy as np
import sys
from music21 import converter, instrument, note, chord
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, BatchNormalization, Dropout, Dense, Activation

sys.path.append("..")
from musiclearn import config

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
midi_dir = pathlib.Path(config.FF_MIDI_DIR)

In [ ]:
def get_notes(midi_dir):
    """ Get all the notes and chords from the midi files in the collection /ff_midi_songs """
    notes = []
    
    files_dir = str(midi_dir) + "/*.mid"
    for file in glob.glob(files_dir):
        midi = converter.parse(file)
        
        print("Parsing %s" % file)
        
        notes_to_parse = None
        
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                 notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    
    """
    notes_file = midi_dir + "/notes"
    with open(notes_file, 'wb') as fp:
        pickle.dump(notes, fp) """
    
    return notes

In [ ]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the neural network """
    sequence_length  = 100
    
    # get all pitch name
    pitchnames = sorted(set(item for item in notes))
    
    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    network_input = []
    network_output = []
    
    # create input sequences and teh corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
        
    n_samples = len(network_input)
    
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_samples, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    
    network_output = keras.utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [ ]:
def lstm_model(network_input, n_vocab):
    """ create the structure of the neural network """
    """
    model = Sequential([
        LSTM(
            512,
            input_shape=(network_input.shape[1], network_input.shape[2]),
            recurrent_dropout=0.3,
            return_sequences=True
        ),
        LSTM(512, return_sequences=True, recurrent_dropout=0.3),
        LSTM(512),
        BatchNormalization(),
        Dropout(0.3),
        Dense(256),
        Activation('relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(n_vocab),
        Activation('softmax')
    ])"""
    
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    return model

In [ ]:
def train_network(notes, epochs = 200, batch_size = 128):
    """ Train a neural network to generate music """
    # get amount of pitch names
    n_vocab = len(set(notes))
    
    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = lstm_model(network_input, n_vocab)
    
    # train the network
    filepath = "saved_weight.hdf5"
    
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        node='min'
    )
    
    callback_list = [checkpoint_cb]
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=batch_size, callbacks=callback_list)

In [ ]:
# extract features
notes = get_notes(midi_dir)

In [ ]:
# train the model
train_network(notes)